# Velkommen til denne programvaren!

Denne notebooken er utviklet i henhold til brukerhistoriene i gruppens bachelorrapport, relatert til bruk av Geopandas for å hente AR50 data.

## Her er funksjonene som kan utføres i programmet:
- Konvertering av GML- eller GDB AR50-datafiler til FGB-format
- Filtrer kolonnene du vil se og sett dem inn i et interaktivt kart
- Filtrer etter bufferradius for å se verdiene i bufferen
- Filtrer etter "områdekode" for å se verdiene innenfor et område

# Importer disse bibliotekene for at programmet skal fungere

In [1]:
import fiona
import os
import pyogrio
import geopandas as gpd
from shapely.geometry import Point
import folium
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

## Kjør denne cellen for at konverteringsfunksjoner skal tre i kraft

In [2]:
def convert_ar50_to_fgb(input_file, output_file):
    """
    Konverterer AR50-data fra GML eller GDB til FlatGeobuf,
    og filtrerer kun ut polygon-geometrier. Feil fanges og logges, slik at
    funksjonen ikke krasjer.
    """
    try:
        ext = os.path.splitext(input_file)[1].lower()

        # Velg lag basert på filtypen
        if ext == ".gml":
            layer_name = "ArealressursFlate"
            if layer_name not in fiona.listlayers(input_file):
                print(f"GML-filen mangler forventet lag '{layer_name}'.")
                return
            gdf = gpd.read_file(input_file, layer=layer_name)

        elif ext == ".gdb":
            layer_name = "ar50"
            layers = fiona.listlayers(input_file)
            if layer_name not in layers:
                print(f"GDB-filen mangler lag '{layer_name}'. Tilgjengelige lag: {layers}")
                return
            gdf = gpd.read_file(input_file, layer=layer_name)

        else:
            print(f"Ugyldig filtype '{ext}'. Støttes kun .gml og .gdb.")
            return

        #  Filtrer kun polygoner/MultiPolygoner
        before = len(gdf)
        gdf = gdf[gdf.geom_type.isin(['Polygon', 'MultiPolygon'])]
        after = len(gdf)
        if after == 0:
            print(f"Ingen Polygon/MultiPolygon-geometrier funnet ({before} → {after}).")
            return

        #  Sørg for at output-mappen finnes
        os.makedirs(os.path.dirname(output_file), exist_ok=True)

        #  Lagre som FlatGeobuf
        gdf.to_file(output_file, driver="FlatGeobuf")
        print(f"Lyktes! Lagret {after} polygoner fra '{input_file}' til '{output_file}'")

    except Exception as e:
        print(f"Feil under konvertering av {input_file}: {e}")
        # returnerer uten å kaste videre




## Begynn å konvertere GML- og GDB-filer til FGB

In [35]:
input_gml = "NO/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/42_25832_ar50_gml.gml"
input_gdb = "NO/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/42_25832_ar50_gdb.gdb"

# Konverter GML til FGB
output_fgb_gml = "NO/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/AR50_FDG/3output_gml.fgb"
convert_ar50_to_fgb(input_gml, output_fgb_gml)

# Konverter GDB til FGB
output_fgb_gdb = "NO/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/AR50_FDG/4output_gdb.fgb"
convert_ar50_to_fgb(input_gdb, output_fgb_gdb)

Feil under konvertering av NO/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/42_25832_ar50_gml.gml: Failed to open dataset (flags=68): NO/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/42_25832_ar50_gml.gml
Feil under konvertering av NO/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/42_25832_ar50_gdb.gdb: Failed to open dataset (flags=68): NO/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/42_25832_ar50_gdb.gdb


## Her kan du skrive inn ønsket kolonne og verdi du vil filtrere etter! Du kan også filtrere etter  postnummer og hvor mange rader du vil vise.

# Vis grensesnittet

In [37]:
def filtering_table_map_buffer_ar50(
    fgb_path, column, center_x, center_y, buffer_radius, filter_value=""
):
    """
    Oppretter et kart basert på AR50-data med partial read.
    Leser kun nødvendig kolonne og geometri.
    Viser alle objekter innenfor bufferområdet med fargekoding etter
    verdien i 'column'. Objekter som delvis ligger innenfor bufferen,
    klippes slik at kun delen innenfor bufferen vises.
    Rader med verdien 98 eller 99 i den valgte kolonnen ekskluderes.
    Dersom filter_value er satt (ikke tom), behold kun rader der column == filter_value.
    """
    # Les inn kun geometri og ønsket kolonne
    gdf = pyogrio.read_dataframe(fgb_path, columns=["geometry", column])
    # Konverter til UTM for metrisk beregning
    gdf = gdf.to_crs(epsg=25832)

    # Lag senterpunkt og buffergéometri
    center_point = Point(center_x, center_y)
    center_point = (
        gpd.GeoSeries([center_point], crs="EPSG:4326")
           .to_crs(epsg=25832)
           .iloc[0]
    )
    buffer_geom = center_point.buffer(buffer_radius)
    buffer_gdf = (
        gpd.GeoDataFrame(geometry=[buffer_geom], crs="EPSG:25832")
           .to_crs(epsg=4326)
    )

    # Klipp ut kun det som ligger innenfor bufferen
    gdf_clipped = gpd.clip(gdf, buffer_geom)
    # Ekskluder verdiene 98 og 99
    gdf_clipped = gdf_clipped[~gdf_clipped[column].isin([98, 99])]

    # Filtrer på valgt verdi hvis oppgitt
    if filter_value not in ("", None):
        gdf_clipped = gdf_clipped[gdf_clipped[column] == filter_value]

    # Konverter tilbake til WGS84 for visning
    table_data = gdf_clipped.to_crs(epsg=4326)

    # Bygg fargekart basert på unike verdier
    unique_values = table_data[column].unique().tolist()
    colors = [
        'red', 'blue', 'green', 'orange', 'purple', 'darkred', 'lightred', 'beige',
        'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white', 'pink',
        'lightblue', 'lightgreen', 'gray', 'black', 'lightgray'
    ]
    color_map = {val: colors[i % len(colors)] for i, val in enumerate(unique_values)}
    print("Fargekart:", color_map)

    # Beregn kartets sentrum fra bufferen
    center_coords = [
        buffer_gdf.geometry.unary_union.centroid.y,
        buffer_gdf.geometry.unary_union.centroid.x
    ]
    m = folium.Map(location=center_coords, zoom_start=12, tiles="cartodb positron")

    # Legg til bufferområdet som eget lag
    folium.GeoJson(
        buffer_gdf,
        name="Bufferområde",
        style_function=lambda x: {"color": "black", "weight": 1, "fillOpacity": 0.0},
    ).add_to(m)

    # Legg til ett GeoJson-lag per unik verdi
    for val in unique_values:
        subset = table_data[table_data[column] == val]
        folium.GeoJson(
            subset,
            name=f"{column}: {val}",
            style_function=lambda feature, val=val: {
                "color": color_map[val],
                "weight": 1,
                "fillOpacity": 0.4
            },
            tooltip=folium.GeoJsonTooltip(fields=[column], aliases=[f"{column}:"])
        ).add_to(m)

    folium.LayerControl().add_to(m)
    return m, table_data


# Funksjon for Komid-filtrering
def filter_table_map_Komid_ar50(ar50_path, filter_column, filter_value, komid_value=None, n_rows=10):
    """
    Leser AR50-data og filtrerer ut rader der:
      - Verdien 98 i filter_column alltid ekskluderes.
      - Dersom filter_value er angitt, behold kun rader med denne verdien.
      - Dersom komid_value er angitt (ikke 0), behold kun rader med denne komid.
    Returnerer:
      1) En GeoDataFrame med de første n_rows radene (kolonner: filter_column, rid, komid, geometry)
      2) Totalt antall rader som matcher filtrene.
    """
    gdf = gpd.read_file(ar50_path)
    if filter_column in gdf.columns:
        gdf = gdf[gdf[filter_column] != 98]

    if filter_value not in ("", None):
        gdf = gdf[gdf[filter_column] == filter_value]

    if komid_value not in (None, 0):
        gdf = gdf[gdf["komid"] == komid_value]

    total_count = len(gdf)
    cols = [c for c in [filter_column, "rid", "komid", "geometry"] if c in gdf.columns]
    return gdf[cols].head(n_rows), total_count


# Bygg brukergrensesnitt

# Filvalg
file_label = widgets.Label(value="Filsti:")
file_path_widget = widgets.Text(
    value="/Users/johannehaakenstad/Bachelor-Filer/Github-KartAI/Johanne/data/AR50/AR50_FDG/output_gdb.fgb",
    layout=widgets.Layout(width='500px')
)
file_done_button = widgets.Button(
    description="Last filsti", button_style='info', layout=widgets.Layout(width='150px')
)
file_output = widgets.Output()
file_box = widgets.VBox([file_label, file_path_widget, file_done_button])

# Filtreringsparametre
filter_column_widget = widgets.Text(value="artype", layout=widgets.Layout(width='500px'))
filter_value_widget  = widgets.Text(value="10",      layout=widgets.Layout(width='500px'))
komid_value_widget   = widgets.Text(value="0",       layout=widgets.Layout(width='500px'))
buffer_radius_widget = widgets.IntText(value=0,       layout=widgets.Layout(width='500px'))
center_x_widget      = widgets.Text(value="7.9956",  layout=widgets.Layout(width='500px'))
center_y_widget      = widgets.Text(value="58.1467", layout=widgets.Layout(width='500px'))
n_rows_widget        = widgets.IntText(value=10,      layout=widgets.Layout(width='500px'))

run_filter_button = widgets.Button(
    description="Utfør filtrering", button_style='info',
    layout=widgets.Layout(width='150px')
)
filter_output = widgets.Output()

filter_widgets_box = widgets.VBox([
    widgets.Label("Filterkolonne:"), filter_column_widget,
    widgets.Label("Filterverdi (la stå tom for filtrering på alle):"), filter_value_widget,
    widgets.Label("Komid (0 = ignorer):"), komid_value_widget,
    widgets.Label("Bufferradius i meter(0 = ignorer):"), buffer_radius_widget,
    widgets.Label("Koordinat X (lon):"), center_x_widget,
    widgets.Label("Koordinat Y (lat):"), center_y_widget,
    widgets.Label("Maks rader:"), n_rows_widget,
    run_filter_button, filter_output
])
filter_widgets_box.layout.display = 'none'

display(widgets.VBox([file_box, file_output, filter_widgets_box]))


# Handler for filvalg
def on_file_done_clicked(b):
    with file_output:
        file_output.clear_output()
        sti = file_path_widget.value.strip()
        if not sti:
            print("Feil: Skriv inn en gyldig filbane.")
            return
        try:
            samp = gpd.read_file(sti).head(50)
            cols = [c for c in samp.columns if c not in ["geometry","arkartstd","kilde"]]
            df = pd.DataFrame({
                "Kolonne": cols,
                "Eksempler": [", ".join(map(str, samp[c].unique()[:5])) for c in cols]
            })
            print("Tilgjengelige kolonner og eksempelverdier:")
            display(df)
            filter_widgets_box.layout.display = ''
        except Exception as e:
            print("Kunne ikke lese fil. Feilmelding:", e)

file_done_button.on_click(on_file_done_clicked)


# Handler for filtrering
def on_run_filter_clicked(b):
    with filter_output:
        filter_output.clear_output()
        print("Kjører filtrering...")

        # Les av inputen
        sti         = file_path_widget.value.strip()
        kol         = filter_column_widget.value.strip()
        val_str     = filter_value_widget.value.strip()
        komid_str   = komid_value_widget.value.strip()
        buf_rad     = buffer_radius_widget.value
        cx_str      = center_x_widget.value.strip()
        cy_str      = center_y_widget.value.strip()
        n_rows      = n_rows_widget.value

        # Validering
        if buf_rad and buf_rad > 0:
            if not kol:
                print("Feil: Fyll inn en kolonne.")
                return
            if not cx_str or not cy_str:
                print("Feil: Fyll inn både X- og Y-koordinater.")
                return
        else:
            if not kol:
                print("Feil: Fyll inn en filterkolonne")
                return

        # Parse filterverdi
        if val_str == "":
            filter_value = ""
        else:
            try:
                filter_value = float(val_str) if "." in val_str else int(val_str)
            except ValueError:
                filter_value = val_str

        # Parse komid
        if komid_str.lower() in ["", "none", "0"]:
            komid_value = None
        else:
            try:
                komid_value = float(komid_str) if "." in komid_str else int(komid_str)
            except ValueError:
                komid_value = komid_str

        # Buffer- eller komid-filtrering
        if buf_rad and buf_rad > 0:
            print("Suksess!")
            try:
                cx, cy = float(cx_str), float(cy_str)
            except ValueError:
                print("Feil: må være tall.")
                return

            try:
                m, table_data = filtering_table_map_buffer_ar50(
                    sti, kol, cx, cy, buf_rad, filter_value
                )
            except Exception as e:
                print("Feil i bufferfiltreringen:", e)
                return

            result_table = table_data.head(n_rows)
            total_count  = len(table_data)

        else:
            print("Suksess!")
            try:
                result_table, total_count = filter_table_map_Komid_ar50(
                    sti, kol, filter_value, komid_value, n_rows
                )
            except Exception as e:
                print("Feil i komid-filtreringen:", e)
                return

            if result_table.crs and result_table.crs != "EPSG:4326":
                result_table = result_table.to_crs("EPSG:4326")

            # Fargekoding for Komid
            unike = result_table[kol].unique().tolist()
            colors = [
                'red','blue','green','orange','purple','darkred','lightred','beige',
                'darkblue','darkgreen','cadetblue','darkpurple','white','pink',
                'lightblue','lightgreen','gray','black','lightgray'
            ]
            fargekart = {v: colors[i % len(colors)] for i, v in enumerate(unike)}

            centrum = result_table.geometry.unary_union.centroid
            m = folium.Map(location=[centrum.y, centrum.x], zoom_start=12, tiles="cartodb positron")
            for v in unike:
                delsett = result_table[result_table[kol] == v]
                folium.GeoJson(
                    delsett,
                    name=f"{kol}: {v}",
                    style_function=lambda feat, v=v: {
                        "color": fargekart[v], "weight": 1, "fillOpacity": 0.4
                    },
                    tooltip=folium.GeoJsonTooltip(fields=[kol], aliases=[f"{kol}:"])
                ).add_to(m)
            folium.LayerControl().add_to(m)

        # Sjekk at det finnes data
        if result_table.empty:
            print("Ingen data funnet for de gitte parametrene.")
            return

        # Vis resultat
        print(f"Hentet totalt {total_count} rader, viser {len(result_table)} rader.")
        display(result_table)
        print("Interaktivt kart:")
        display(m)

run_filter_button.on_click(on_run_filter_clicked)
